# Data

In [ ]:
performance_data = {
    "gpt-4-turbo": {
        "targeted_safety_risks": (-3.5434782608695654, 0.0),
        "system_and_operational_risks": (-2.4456521739130435, 0.0),
        "content_safety_risks": (-1.5978260869565217, 0.0),
        "societal_risks": (-2.1847826086956523, 0.0),
        "legal_and_rights_related_risks": (-2.3369565217391304, 0.0),
        "efficiency": (7.391304347826087, 0.0),
        "goal": (7.163043478260869, 0.0),
        "overall_score": (0.3493788819875778, 0.0),
        "setting_num": (92.0, 0.0),
        "episode_count": (92.0, 0.0),
    },
    "gpt-3.5-turbo": {
        "targeted_safety_risks": (-4.706521739130435, 0.0),
        "system_and_operational_risks": (-3.608695652173913, 0.0),
        "content_safety_risks": (-2.402173913043478, 0.0),
        "societal_risks": (-2.989130434782609, 0.0),
        "legal_and_rights_related_risks": (-2.989130434782609, 0.0),
        "efficiency": (5.75, 0.0),
        "goal": (5.336956521739131, 0.0),
        "overall_score": (-0.8012422360248445, 0.0),
        "setting_num": (92.0, 0.0),
        "episode_count": (92.0, 0.0),
    },
}

ratio_data = {
    "gpt-4-turbo": {
        "targeted_safety_risks": (0.5434782608695652, 0.0),
        "system_and_operational_risks": (0.45652173913043476, 0.0),
        "content_safety_risks": (0.2717391304347826, 0.0),
        "societal_risks": (0.358695652173913, 0.0),
        "legal_and_rights_related_risks": (0.3804347826086957, 0.0),
        "efficiency": (0.0, 0.0),
        "goal": (0.0, 0.0),
        "overall_score": (0.5543478260869565, 0.0),
        "setting_num": (92.0, 0.0),
        "episode_count": (92.0, 0.0),
    },
    "gpt-3.5-turbo": {
        "targeted_safety_risks": (0.6739130434782609, 0.0),
        "system_and_operational_risks": (0.6521739130434783, 0.0),
        "content_safety_risks": (0.40217391304347827, 0.0),
        "societal_risks": (0.5, 0.0),
        "legal_and_rights_related_risks": (0.5, 0.0),
        "efficiency": (0.0, 0.0),
        "goal": (0.0, 0.0),
        "overall_score": (0.6847826086956522, 0.0),
        "setting_num": (92.0, 0.0),
        "episode_count": (92.0, 0.0),
    },
}

# Main performance table

In [ ]:
def generate_latex_table(performance, models, wrap=True):
    # Define the table dimensions and labels
    dimensions = {
        "targeted_safety_risks": "Targeted Safety Risks & [-10, 0]",
        "system_and_operational_risks": "System and Operational Risks & [-10, 0]",
        "content_safety_risks": "Content Safety Risks & [-10, 0]",
        "societal_risks": "Societal Risks & [-10, 0]",
        "legal_and_rights_related_risks": "Legal and Rights-Related Risks & [-10, 0]",
        "efficiency": "Efficiency & [0, 10]",
        "goal": "Goal Completion & [0, 10]",
    }
    models_mapping = {
        "gpt-4-turbo": "GPT-4",
        "gpt-3.5-turbo": "GPT-3.5",
        "llama3": "Llama3",
        "Claude3.5": "Claude3.5",
    }

    # Start building the LaTeX code
    # Start building the LaTeX code
    if wrap:
        latex_code = """
\\begin{wraptable}[13]{r}{8.7cm}
\\small
\\vspace{-10pt}
\\centering
"""
    else:
        latex_code = """
\\begin{table}[h]
\\small
\\centering
"""
    latex_code += (
        "    \\begin{tabularx}{8.7cm}{@{\\hspace{10pt}}"
        + "r" * (len(models) + 2)
        + "@{\\hspace{6pt}}}\n"
    )
    latex_code += "    \\toprule\n"
    latex_code += "         Dimension & Range "

    # Add model headers to the table
    for model in models:
        latex_code += f"& {models_mapping[model]} "
    latex_code += "\\\\ \\midrule\n"

    # Populate the table with data
    for dim_key, dim_label in dimensions.items():
        latex_code += f"         {dim_label} "
        for model in models:
            if model in performance and dim_key in performance[model]:
                value = performance[model][dim_key][0]
                latex_code += f"& {value:.2f} "
            else:
                latex_code += "& "
        latex_code += "\\\\ \n"

    # Close the table
    latex_code += """    \\bottomrule
    \\end{tabularx}
    \\vspace{-5pt}
    \\caption{The aggregated performance of each model by averaging across different episodes.}
    \\label{tab:model_performance}
"""
    if wrap:
        latex_code += "\\end{wraptable}\n"
    else:
        latex_code += "\\end{table}\n"
    return latex_code

In [ ]:
latex_code = generate_latex_table(
    performance=performance_data,
    models=["gpt-4-turbo", "gpt-3.5-turbo", "llama3", "Claude3.5"],
    wrap=False,
)
print(latex_code)

### Risk ratio graph

In [ ]:
## risk ratio graph
from typing import Dict, Tuple

import matplotlib.pyplot as plt
import seaborn as sns


def generate_bar_graph(
    ratio_data: Dict[str, Dict[str, Tuple[float, float]]], save_path: str = ""
) -> None:
    # Define the categories to include in the graph
    categories = [
        "overall_score",
        "targeted_safety_risks",
        "system_and_operational_risks",
        "content_safety_risks",
        "societal_risks",
        "legal_and_rights_related_risks",
    ]
    category_acronyms = {
        "overall_score": "Overall",
        "targeted_safety_risks": "Tar.R",
        "system_and_operational_risks": "Sys.R",
        "content_safety_risks": "Con.R",
        "societal_risks": "Soc.R",
        "legal_and_rights_related_risks": "Legal.R",
    }

    # Initialize data containers for plotting
    models = list(ratio_data.keys())
    category_values = {category: [] for category in categories}

    # Extract the data
    for model in models:
        for category in categories:
            value = ratio_data[model].get(category, (0.0, 0.0))[0]
            category_values[category].append(value)

    # Set up Seaborn's style
    sns.set(style="whitegrid")

    # Create a new figure
    fig, ax = plt.subplots(figsize=(12, 5))  # Smaller figure size

    # Define color palette (blues)
    palette = sns.color_palette("Blues", len(models))

    # Plot bars for each model
    bar_width = 0.35  # Width of the bars
    index = range(len(categories))  # X locations for the groups

    for i, model in enumerate(models):
        bars = ax.bar(
            [p + i * bar_width for p in index],
            [category_values[cat][i] for cat in categories],
            bar_width,
            label=model.replace("_", "-").capitalize(),
            color=palette[i],
        )
        # Add numbers on top of bars
        for bar in bars:
            yval = bar.get_height()
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                yval + 0.01,
                f"{yval:.2f}",
                ha="center",
                va="bottom",
                fontsize=10,
            )

    # Labels, title, and legend
    ax.set_ylabel("Risk Ratio", fontsize=12, fontweight="bold")
    ax.set_xticks([p + (bar_width * (len(models) - 1) / 2) for p in index])
    ax.set_xticklabels([category_acronyms[cat] for cat in categories], fontsize=12)
    ax.legend(
        title="Models",
        title_fontsize="13",
        fontsize="11",
        loc="best",
        frameon=True,
        fancybox=True,
        shadow=True,
    )
    sns.despine()
    # Improve visual spacing
    if save_path:
        plt.savefig(save_path, format="pdf", bbox_inches="tight")
    plt.tight_layout()
    plt.show()

In [ ]:
generate_bar_graph(ratio_data=ratio_data, save_path="./risk_ratio_graph.pdf")

### Episodes with no target risks but other risks